- Goal
    - Create synthetic data using IDM
    - Learn parameters of that data using particle filtering
    - Aliter 7 Feb: Learn using CEM idea, fitness function and then distb and then sample
- Learning
    - Need at least 2 vehicles so that there is a neighbor in the front
    - Otherwise src/1d/driver/lane_follower_driver errors
    - That is why the `AutomotiveDrivingModels/doc/1DMobius` stuff does 
    not work with 1 car only (needs at least 2 cars)
    - The tutorial does not work with `gen_straight_roadway` because that
    generates a roadway of type AutomotiveDrivingModels.roadway as opposed to
    AutomotiveDrivingModels.StraightRoadway
- Open question
    - IDM won't work unless there is a car in front (errors saynig nothing in sight)
    - We are focusing on learning the params of the second car here. Is that sound sensible?
- Flow of code
    - Call the required `usings`
    - Define functions required
    - Actual running things
        - Generate true trajectory
        - Generate a set of particles uniformly between sensible range of values
        - Compute fitness, sort and select numtop
        - Fit a distribution over this
        - Resample particles
- Scenario
    - Slower car in front (car 1)
    - Faster car behind (car 2)
    - We want to estimate params of car 2
- Feb 10: Adding timegap_des as our 2nd param in the 2 car, 1D, IDM scenario
    - Make code capable of handling 2d param. So far, had only scalar param
    i.e v_des
    - Fitting 2D distributions is required now
- Feb 22
    - the `rec` generated using `simulate` is a devious monster
    - It stores the last timestep in the 1st entry and first timestep in last entry
    - Be careful

In [1]:
using AutomotiveDrivingModels
using AutoViz
using Reel
using Interact
using StatsBase # For random particle generation

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
# 1 lane, 1000 m roadway
# roadway = gen_straight_roadway(1,1000.0) # Does not work with the below because AutomotiveDrivingModels.Roadway
roadway = StraightRoadway(1000.0); # AutomotiveDrivingModels.StraightRoadway type

In [3]:
"""
Initialize a scene given positions and velocities of 2 cars.

Usage:
`scene = init_scene(car1.s,car1.v,car2.s,car2.v)

Returns:
- `scene`

BREAKING POSS:
- Hard coded for 2 vehicles
"""
# Function: Initialize the scene
    # Note: Vehicle def 1 (leader) is the one we learn params for
    # This is hard-coded in the gen_traj method by accessing first elem of scene vector
# The returned thing here is an array of entities i.e. cars and you can query those to get pos, vel
function init_scene(pos1=50.0,vel1=12.0,pos2=10.0,vel2=10.0)
    scene = Scene1D()
    
    # First arg to State1D is pos, 2nd arg is velocity
    push!(scene, Entity(State1D(pos1, vel1), VehicleDef(), 1))
    push!(scene, Entity(State1D(pos2, vel2), VehicleDef(), 2))
    return scene
end

init_scene

In [49]:
"""
Generate a ground truth trajectory.

Usage:
pos_ground_truth, rec_ground_truth = gen_traj([20.0 0.1])

Arguments:
- `particle` Ground truth IDM parameter values
- `nticks` Number of timesteps simulated default 100
- `timestep` Duration of a step default 0.1

Returns:
- `X` Array of length `nticks` containing position of car1 with X[1] being 
position after 1 timestep from initialization
- `rec` Queuerecord of scenes which can be accessed by `rec.frames` which is
of length `nticks+1`. Note that `rec.frames[nticks+1] contains the initialization scene
and rec.frames[1] contains the final scene. Thus info is contained in backward indexing.

Breaking possibilities:
- Hard coded to work with 2 cars by calling only 2 models
- Only returns position of car1 in return value `X`
"""
function gen_traj(particle;nticks=100,timestep=0.1)
    scene = init_scene()
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(v_des=particle[1],σ = particle[2])
    models[2] = IntelligentDriverModel(v_des=12.0)

    # Simulate for nticks (default 100) time steps
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    # Extract the position and velocity of nticks timesteps
    # X stores this as 2d array. Timestep is the row, col 1 is pos,vel is pos2
    n_cars = scene.n
    n_ticks = nticks
    X = Array{Float64}(n_ticks, 1)

    for t in 1:n_ticks
        f = rec.frames[n_ticks - t + 1]
        
        # BAD: 2 cars in scene therefore loop has only 1 elem
        for c in 1:1 #Was 2:sc
            s = f.entities[c].state
            X[t, 1] = s.s #position
        end
    end
    return X, rec
end

gen_traj

In [4]:
"""
Hallucinate a step given positions and velocities of 2 cars and a candidate particle.

Usage:
`pos_ground_truth,rec_ground_truth = gen_traj([20.0 0.1])`
`frame_of_interest = rec_ground_truth.frames[101]`
`test = hallucinate_a_step(frame_of_interest.entities[1].state,
                            frame_of_interest.entities[2].state,[20.0 0.1])`

Arguments:
- `car1` The leader car whose parameters we are trying to learn
- `car2` The follower car
- `particle` The candidate IDM params eg: [20.0 0.1]

Returns:
- `X` Position of the leader car after hallucinating one step under the candidate particle

BREAKING POSSIBILITIES
- If Number of cars in scene and number of models are ever different
- Timestep and nticks are hardcoded within the function
- Artificial creation of scene in the beginning is scalability issue
- Only returns position of leader car
"""
function hallucinate_a_step(car1,car2,particle)
    scene = init_scene(car1.s,car1.v,car2.s,car2.v)
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(v_des=particle[1],σ = particle[2])
    models[2] = IntelligentDriverModel(v_des=12.0)

    # Simulate for nticks time steps
    nticks = 1
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    # Extract the position and velocity of nticks timesteps
    # X stores this as 2d array. Timestep is the row, col 1 is pos,vel is pos2
    n_cars = scene.n
    n_ticks = nticks
    X = Array{Float64}(n_ticks, 1)

    for t in 1:n_ticks
        f = rec.frames[n_ticks - t + 1]
        
        # BAD: 2 cars in scene therefore loop has only 1 elem
        for c in 1:1 #Was 2:sc
            s = f.entities[c].state
            X[t, 1] = s.s #position
        end
    end
    return X
end

hallucinate_a_step

In [126]:
"""
Calculate log likelihood of a candidate particle over a trajectory by running hallucination
over given ground truth queuerecord and ground truth true position array.

Usage:
`pos_ground_truth,rec_ground_truth = gen_traj([20.0 0.1])`
`calc_traj_log_likelihood([20.0 0.01],rec_ground_truth,pos_ground_truth)`

Arguments:
- `particle` Candidate particle eg [20.0 0.1]
- `rec_ground_truth` Queuerecord with ground truth scenes
- `pos_ground_truth` Ground truth with position array

Returns:
- `log_lkhd` log likelihood of truth given candidate particle 

This function calls:
- `hallucinate_a_step`

BREAKING POSSIBILITIES
- particle[2] has been hardcoded as the variance
- Hardcoded to work with 2 cars in the scene
- Timestep harcoded as 0.1 for std dev calculation
- https://stattrek.com/random-variable/transformation.aspx
WHAT SHOULD THE LOG LKHD BE INITIALIZED TO? CURRENTLY INITIALIZED TO 0
"""
function calc_traj_log_likelihood(particle,rec_ground_truth,pos_ground_truth)
    std_dev_acc = particle[2]
    
    # hack to avoid the std_dev_pos become negative and error Normal distb
    if std_dev_acc < 0 std_dev_acc = 0.1 end

    log_lkhd = 0
    f_end_num = length(rec_ground_truth.frames)
    for t in 1:f_end_num-1
        f = rec_ground_truth.frames[f_end_num - t + 1]

        hpos = hallucinate_a_step(f.entities[1].state,f.entities[2].state,particle)
        # Ground truth pos
        trupos = pos_ground_truth[t]

        timestep = 0.1
        std_dev_pos = timestep*timestep*std_dev_acc
        
        log_lkhd+=logpdf(Normal(hpos[1],std_dev_pos),trupos[1])
    end
    return log_lkhd
end

calc_traj_log_likelihood

In [336]:
"""
Update particles over one step of ground truth

Usage:
```julia REPL
# Sanity check update_particles_one_step
# Generate particles
num_p = 5
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:50.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)

# Arrange the particles as follows
    # Every column is a different particle
    # Row 1 has v_des and row 2 has sigma
init_particles = vcat(v_particles',sig_particles')
@show init_particles

# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj([25.0 0.6])

new_p_mat = update_particles_one_step(init_particles,rec_ground_truth.frames[101],pos_ground_truth[1])
```

Arguments:
-`particle_matrix`
-`f` A scene
-`trupos` Single element array containing ground truth position one timestep after scene f

Issues:
-Assumes that element in 2nd row is going to be std_dev
- Timestep hardcoded to be 0.1
"""
function update_particles_one_step(particle_matrix,f,trupos)
    num_p = size(particle_matrix)[2]
    timestep = 0.1
    
    lkhd_vec = Array{Float64}(num_p)
    for i in 1:num_p
        candidate_particle = particle_matrix[:,i]
        std_dev_acc = candidate_particle[2]
            
        # hack to avoid the std_dev_pos become negative and error Normal distb
        if std_dev_acc < 0 std_dev_acc = 0.1 end

        std_dev_pos = timestep*timestep*std_dev_acc
            
        hpos = hallucinate_a_step(f.entities[1].state,f.entities[2].state,candidate_particle)
        
        lkhd_vec[i] = pdf(Normal(hpos[1],std_dev_pos),trupos[1])
        
    end

    p_weight_vec = weights(lkhd_vec./sum(lkhd_vec))

    # Sample num_p new particles according to weights
    idx = sample(1:num_p,p_weight_vec,num_p)
    new_particle_matrix = particle_matrix[:,idx] #Careful that idx is (size,1) and not (size,2)
    
    return new_particle_matrix
end

update_particles_one_step

In [343]:
"""
Run particle filter over a trajectory

Usage:
```julia REPL
# Generate particles
num_p = 1000
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:30.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)

# Arrange the particles as follows
    # Every column is a different particle
    # Row 1 has v_des and row 2 has sigma
init_particles = vcat(v_particles',sig_particles')
# @show init_particles

# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj([25.0 0.6])

pf_traj(init_particles,rec_ground_truth,pos_ground_truth)
```

Arguments:
-`particle_matrix` Initial set of particles
-`rec_ground_truth` Queuerecord containing ground truth scenes
-`pos_ground_truth` Array of ground truth positions

Issues:
-`timestep` hardcoded
- If all the particles are exactly identical, fitting will not work
"""
function pf_traj(particle_matrix,rec_ground_truth,pos_ground_truth)
    num_p = size(particle_matrix)[2]
    
    log_lkhd = 0
    f_end_num = length(rec_ground_truth.frames)
    
    # loop over the trajectory step by step
    
        # loop over the set of particles
            # assign likelihood to a particle
        
        # Make weight vector
        # Resample from weight vector
    timestep = 0.1 # TODO REMOVE HARDCODING
    
    old_p_mat = particle_matrix
    
    for t in 1:f_end_num-1
#         @show t
        f = rec_ground_truth.frames[f_end_num - t + 1]
        trupos = pos_ground_truth[t]
        
        new_p_mat = update_particles_one_step(old_p_mat,f,trupos)
#         @show new_p_mat
        old_p_mat = new_p_mat
    end
    
    @show old_p_mat
    @show fit(MvNormal,old_p_mat)
end

pf_traj

In [127]:
"""
Generate a new sample set of particles from old set for CEM

Usage:
```julia REPL
# Generate particles
num_p = 100
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:50.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)

# Arrange the particles as follows
    # Every column is a different particle
    # Row 1 has v_des and row 2 has sigma
init_particles = vcat(v_particles',sig_particles')
# @show init_particles
distb = fit(MvNormal,init_particles)
@show distb

# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj([25.0 0.6])

# Generate new particles
p_old = init_particles
p_dist = []  # TODO: Is this empty init fine for a distribution
# @show p_old
for i in 1:20
    if i%10==0 @show i end
    p_new, p_dist = new_p_from_old(p_old,num_p,rec_ground_truth,pos_ground_truth,elite_fraction_percent=20)
    p_old = p_new
end
@show p_dist
```

Arguments:
- `particle_matrix` Matrix with each column containing a candidate particle and row containing param value
- `num_p` Number of particles i.e. number of columns of `particle_matrix`
- `rec_ground_truth` Queuerecord containing array of ground truth scenes generated using `gen_traj`
- `pos_ground_truth` Array of ground truth positions generated using `gen_traj`
- `elite_fraction_percent` Percentage of particles to be selected as elite for fitting new distribution

Returns:
- `new_p_matrix` Matrix containing `num_p` particles sampled from distribution fitted from elite particles
- 'p_distribution` Distribution obtained from final set of particles

Improvements needed:
- TODO: MAYBE DOING THIS IN PLACE WILL BE MORE EFFICIENT RATHER THAN RETURN WHOLE MATRIX
"""
function new_p_from_old(particle_matrix,num_p,rec_ground_truth,pos_ground_truth;elite_fraction_percent=20)
    assert(size(particle_matrix)[2]==num_p) #check numcols same as num particles
    
    p_traj_lkhd = Array{Float64}(num_p) # for storing loglikhd for each particle
    
    for i in 1:num_p
        candidate_particle = particle_matrix[:,i]
        p_traj_lkhd[i] = calc_traj_log_likelihood(candidate_particle,rec_ground_truth,pos_ground_truth)
    end

    # We don't care about the actual error values
    # We just care about the numtop of them

    # Sort the traj such that highest likelihood is at the top
        # rev=true does this for us i.e sort in decreasing order
    sortedidx = sortperm(p_traj_lkhd,rev=true)
    
    numtop = convert(Int64,ceil(num_p*elite_fraction_percent/100.0))
    best_particles = particle_matrix[:,sortedidx[1:numtop]] # elite selection
    p_distribution = fit(MvNormal,best_particles) # fit distb using elites
    new_p_matrix = rand(p_distribution,num_p) # sample num_p new particles from distb

    return new_p_matrix, p_distribution
end

new_p_from_old

In [134]:
# TEST THE CEM PF APPROACH
# Generate particles
num_p = 100
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:50.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)

# Arrange the particles as follows
    # Every column is a different particle
    # Row 1 has v_des and row 2 has sigma
init_particles = vcat(v_particles',sig_particles')
# @show init_particles
distb = fit(MvNormal,init_particles)
@show distb

# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj([25.0 0.6])

# Generate new particles
p_old = init_particles
p_dist = []  # TODO: Is this empty init fine for a distribution
# @show p_old
for i in 1:20
    if i%10==0 @show i end
    p_new, p_dist = new_p_from_old(p_old,num_p,rec_ground_truth,pos_ground_truth,elite_fraction_percent=20)
    p_old = p_new
end
@show p_dist

distb = FullNormal(
dim: 2
μ: [28.71, 0.506]
Σ: [129.686 0.02574; 0.02574 0.065164]
)

i = 10
i = 20
p_dist = FullNormal(
dim: 2
μ: [24.9958, 0.322712]
Σ: [0.00343399 0.000369269; 0.000369269 6.35364e-5]
)



FullNormal(
dim: 2
μ: [24.9958, 0.322712]
Σ: [0.00343399 0.000369269; 0.000369269 6.35364e-5]
)


In [284]:
"""
Update particles by likelihood and then resampling
Should not work because logliklihoods are negative thus how will you weight particles?

Arguments:
- `particle_matrix`
- `num_p`
- `rec_ground_truth`
- `pos_ground_truth`

Returns:
- `new_particle_matrix`

Usage:

Issues:
"""
function update_particles(particle_matrix,num_p,rec_ground_truth,pos_ground_truth)
    assert(size(particle_matrix)[2]==num_p) #check numcols same as num particles
    
    # Calculate the likelihood
    p_traj_loglkhd = Array{Float64}(num_p) # for storing loglikhd for each particle
    
    for i in 1:num_p
        candidate_particle = particle_matrix[:,i]
        p_traj_loglkhd[i] = calc_traj_log_likelihood(candidate_particle,rec_ground_truth,pos_ground_truth)
    end
    @show p_traj_loglkhd
    
    p_traj_lkhd = exp.(p_traj_loglkhd)
    @show p_traj_lkhd
    
    # Convert likelihood to weights
    p_weight_vec = weights(p_traj_lkhd./sum(p_traj_lkhd))
    @show p_weight_vec
    
    # Sample num_p new particles according to weights
    idx = sample(1:num_p,p_weight_vec,num_p)
    new_particle_matrix = particle_matrix[:,idx] #Careful that idx is (size,1) and not (size,2)
    
    return new_particle_matrix
end

update_particles

In [285]:
# TEST THE VANILLA PF
# Generate particles
num_p = 3
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:50.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)

# Arrange the particles as follows
    # Every column is a different particle
    # Row 1 has v_des and row 2 has sigma
init_particles = vcat(v_particles',sig_particles')
@show init_particles
distb = fit(MvNormal,init_particles)
@show distb

# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj([25.0 0.6])

# Generate new particles
p_old = init_particles
p_dist = []  # TODO: Is this empty init fine for a distribution
# @show p_old
for i in 1:1
    if i%10==0 @show i end
    p_new = update_particles(p_old,num_p,rec_ground_truth,pos_ground_truth)
    p_old = p_new
end
@show p_old

init_particles = [50.0 23.0 16.0; 0.4 1.0 0.9]
distb = FullNormal(
dim: 2
μ: [29.6667, 0.766667]
Σ: [214.889 -3.61111; -3.61111 0.0688889]
)

p_traj_loglkhd = [182.073, 346.638, -556.286]
p_traj_lkhd = [1.18441e79, 3.4902e150, 2.55881e-242]
p_weight_vec = [3.39354e-72, 1.0, 0.0]
p_old = [23.0 23.0 23.0; 1.0 1.0 1.0]


2×3 Array{Float64,2}:
 23.0  23.0  23.0
  1.0   1.0   1.0

# Visualize

In [ ]:
# Function: Return rec corresponding to generated traj
    # Will help visualizatoin
    # Calls init_scene
# Might be useful later
    # models[2] = IntelligentDriverModel(v_des=particle[1],s_min=particle[2],T=particle[3])
function gen_rec4vid(particle;nticks=100,timestep=0.1)
    scene = init_scene()
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(v_des=particle[1],σ = particle[2])
    models[2] = IntelligentDriverModel(v_des=12.0)

    # Simulate for nticks (default 100) time steps
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    return rec
end

In [ ]:
# overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true,
#         pos=VecE2(veh.state.s-0.7, 3)) for veh in scene];
# render(scene, roadway, overlays, cam=cam, canvas_height=100)

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
true_rec = gen_rec4vid([20.0 0.1],nticks=100)
rec = true_rec
@manipulate for frame_index in 1 : nframes(rec)
    render(rec[frame_index-nframes(rec)], roadway, cam=cam, canvas_height=100)
end

# LEARNING AND EXPERIMENTATION

In [ ]:
# LEARNING ABOUT MULTIDIM DISTB
# Test: Generate samples for a 2d distb
d2 = MvNormal(2,2.0) # first arg shows dimension, second shows std dev
qw = rand(d2,6) # Will generate 6 samples i.e. 6 columns

# Test: Fit 2d distribution
dx = Normal()
dy = Normal(2,1.0)
x = rand(dx,100)
y = rand(dy,100)

# Matrix with each column being a sample
# Total columns is total number of samples
# Total rows is number of parameters
# All entries in a row contain value from same param eg:v_des
data_matrix = vcat(x',y')
fit(MvNormal,data_matrix)

In [ ]:
using PyPlot

In [ ]:
num_samples = 2000
y1 = rand(Normal(10.0,5.0),num_samples)
y2 = rand(Normal(2.0,1.0),num_samples)
plot(1:num_samples,y1)
plot(1:num_samples,y2)

In [ ]:
using StatPlots

In [ ]:
StatPlots.plot(Normal(3,5),linewidth=4,size=(2500,2500))

In [ ]:
roadway = gen_straight_roadway(2,1000.0);

In [ ]:
scene = Scene1D()
push!(scene, Entity(State1D(10.0,  8.0), VehicleDef(), 1))
push!(scene, Entity(State1D(50.0, 12.5), VehicleDef(), 2))

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, pos=VecE2(veh.state.s-0.7, 3)) for veh in scene]
render(scene, roadway, overlays, cam=cam, canvas_height=100)

In [ ]:
models = Dict{Int64, DriverModel}()
models[1] = Tim2DDriver(0.1) # always produce zero acceleration
models[2] = Tim2DDriver(0.1) # default IDM with a desired speed of 12 m/s

nticks = 100
timestep = 0.1
rec = QueueRecord(Vehicle1D, nticks+1, timestep)
simulate!(rec, scene, roadway, models, nticks)

In [286]:
using Base.Test
@test 1==1
@test 1==0